In [1]:
%config Completer.use_jedi = False

In [2]:
pip install pandas scikit-learn tensorflow mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\DeepL\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import mediapipe as mp
import os
import cv2

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [5]:
# Set the directory of the dataset
dataset_directory = 'SigNN Character Database\\'

In [2]:
# List containing the target signs
targets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

In [7]:
# Create a temporary dataset containing the landmarks of all images in the dataset
tmp_dataset = []

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5) as hands:

    for target in targets:
        for dirpath, dirnames, filenames in os.walk(dataset_directory + target):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                frame = cv2.imread(filepath)

                results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                if results.multi_hand_landmarks:
                    hand_landmarks = []
                    for hand_landmark in results.multi_hand_landmarks[0].landmark:
                        hand_landmarks.extend([hand_landmark.x, hand_landmark.y, hand_landmark.z])
                    
                    tmp_dataset.append((*hand_landmarks, target))


In [8]:
# Labels for the variables of the hand landmarks
landmark_labels = []
for i in range(21):
    for var in ('xyz'):
        label = var + "_" + str(i)
        landmark_labels.append(label)

landmark_labels.append('sign')

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(tmp_dataset, columns=landmark_labels)
df.head()

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,sign
0,0.464909,0.708310,-5.075436e-07,0.555220,0.661323,-0.030122,0.611244,0.556989,-0.037128,0.628575,...,0.420475,0.517954,-0.068963,0.438464,0.582875,-0.067767,0.449238,0.623481,-0.053324,A
1,0.360279,0.952744,-5.851731e-07,0.447779,0.914562,-0.023837,0.508625,0.819380,-0.027467,0.534209,...,0.325673,0.751582,-0.054271,0.335584,0.815995,-0.046663,0.346112,0.858840,-0.028027,A
2,0.428278,0.524772,-2.572420e-07,0.484400,0.516362,-0.018880,0.532799,0.455504,-0.020887,0.545799,...,0.438880,0.360585,-0.026516,0.432223,0.408768,-0.022789,0.426714,0.444198,-0.011816,A
3,0.425652,0.524406,-2.628151e-07,0.481915,0.513582,-0.018556,0.529754,0.453477,-0.020464,0.542318,...,0.435767,0.358213,-0.026046,0.429896,0.405734,-0.022134,0.424611,0.441838,-0.011048,A
4,0.418433,0.523851,-2.631909e-07,0.474829,0.514101,-0.018966,0.523636,0.455153,-0.021238,0.537295,...,0.429258,0.359092,-0.025818,0.424016,0.406955,-0.022319,0.419283,0.443150,-0.011527,A


In [11]:
# Save the DataFrame as a .csv file
df.to_csv('ASL_data.csv', index=False)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X = df.drop(['sign'],axis=1).values
y = df['sign'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=63, input_shape=[63]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=24, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 63)                4032      
                                                                 
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8192      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
y_train_encoded = pd.get_dummies(y_train, prefix='sign')
y_test_encoded = pd.get_dummies(y_test, prefix='sign')
y_train_encoded.head(3)

,sign_A,sign_B,sign_C,sign_D,sign_E,sign_F,sign_G,sign_H,sign_I,sign_K,...,sign_P,sign_Q,sign_R,sign_S,sign_T,sign_U,sign_V,sign_W,sign_X,sign_Y
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [18]:
history = model.fit(
    X_train,
    y_train_encoded,
    validation_data=(X_test, y_test_encoded),
    epochs=100,
    callbacks=[early_stop]
)

Epoch 1/100
207/207 [==============================] - 3s 8ms/step - loss: 2.4514 - accuracy: 0.2125 - val_loss: 1.2834 - val_accuracy: 0.5336
Epoch 2/100
207/207 [==============================] - 1s 7ms/step - loss: 1.1755 - accuracy: 0.5601 - val_loss: 0.8472 - val_accuracy: 0.6669
Epoch 3/100
207/207 [==============================] - 1s 7ms/step - loss: 0.8218 - accuracy: 0.6828 - val_loss: 0.5480 - val_accuracy: 0.7638
Epoch 4/100
207/207 [==============================] - 1s 7ms/step - loss: 0.6832 - accuracy: 0.7475 - val_loss: 0.5308 - val_accuracy: 0.8207
Epoch 5/100
207/207 [==============================] - 1s 6ms/step - loss: 0.5614 - accuracy: 0.7966 - val_loss: 0.4209 - val_accuracy: 0.8631
Epoch 6/100
207/207 [==============================] - 1s 6ms/step - loss: 0.4827 - accuracy: 0.8281 - val_loss: 0.3265 - val_accuracy: 0.8789
Epoch 7/100
207/207 [==============================] - 1s 7ms/step - loss: 0.4374 - accuracy: 0.8422 - val_loss: 0.2906 - val_accuracy: 0.8898

In [19]:
model.save('saved_model/my_model')

In [20]:
test_x = X_train[0].reshape((1,-1))
test_y = y_train[0]

pred_y = targets[np.argmax(model.predict(test_x))]

#print(test_y == pred_y)
print(test_x.shape)

1/1 [==============================] - 0s 67ms/step
(1, 63)


In [21]:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5) as hands:
    
    frame = cv2.imread('SigNN Character Database\\A\\1.jpg')

    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        hand_landmarks = []
        for hand_landmark in results.multi_hand_landmarks[0].landmark:
            hand_landmarks.extend([hand_landmark.x, hand_landmark.y, hand_landmark.z])

    print(targets[np.argmax(model.predict(np.array([hand_landmarks])))])

1/1 [==============================] - 0s 12ms/step
B
